# Mantain Conversation History

## Install OpenAI and LangChain dependencies

In [ ]:
!pip install langchain==0.1.12
!pip install langchain-openai==0.0.8

## Load OpenAI API Credentials

Save the key in 'api_credentials.yml'.

In [3]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [4]:
import yaml

with open('api_credentials.yml', 'r') as file:
    api_creds = yaml.safe_load(file)

In [5]:
api_creds.keys()

dict_keys(['openai_key'])

In [6]:
import os

os.environ['OPENAI_API_KEY'] = api_creds['openai_key']

## Load ChatGPT LLM Libraries

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [8]:
model = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.0) # choose the model

##  Build a conversation chain

In [ ]:
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [ ]:
# create prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Act as a helpful AI Assistant"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
# k=3 stores last 3 conversations between you and ChatGPT
memory = ConversationBufferWindowMemory(k=3, return_messages=True)

In [ ]:
memory.load_memory_variables({}) # shows the conversation history

{'history': []}

In [ ]:
from operator import itemgetter
# creates the conversation chain
chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables)
        |
        itemgetter("history")
    )
    |
    prompt
    |
    model
)

In [ ]:
user_input = {'input': 'What are the first four colors of a rainbow'}
response = chain.invoke(user_input)
response.content

'The first four colors of a rainbow are red, orange, yellow, and green.'

In [ ]:
memory.load_memory_variables({})

{'history': []}

In [ ]:
# remember to save your conversation to the memory
memory.save_context(user_input, {"output": response.content})
memory.load_memory_variables({}) # remembers the conversation history

{'history': [HumanMessage(content='What are the first four colors of a rainbow'),
  AIMessage(content='The first four colors of a rainbow are red, orange, yellow, and green.')]}

In [ ]:
user_input = {'input': 'And the last 3?'}
response = chain.invoke(user_input) # uses history of the past conversation to give a better response
response.content

'The last three colors of a rainbow are blue, indigo, and violet.'

In [ ]:
memory.save_context(user_input, {"output": response.content})
memory.load_memory_variables({})

{'history': [HumanMessage(content='What are the first four colors of a rainbow'),
  AIMessage(content='The first four colors of a rainbow are red, orange, yellow, and green.'),
  HumanMessage(content='And the last 3?'),
  AIMessage(content='The last three colors of a rainbow are blue, indigo, and violet.')]}